# Notebook to compare Penn CHIME fits against all parameter fits

This notebook compares the implementation of the Penn CHIME SIR model against this implementation of the SIR model (needed for fitting).
Comparisons are implemented for default parameters with and without social distancing policies.

**TL;DR:** They agree.

## Imports

In [ ]:
from os import environ
from datetime import date

from pandas import DataFrame
from numpy import zeros

from penn_chime.model.parameters import Parameters, Disposition
from penn_chime.model.sir import (
    Sir,
    sim_sir,
    calculate_dispositions,
    calculate_admits,
    calculate_census,
)

from models import sir_step, sihr_step, one_minus_logistic_fcn, FitFcn

In [ ]:
COLS_TO_COMPARE = [
    "susceptible",
    "infected",
    "recovered",
    "hospitalized_new",
    "hospitalized",
]
COLUMN_MAP = {
    "hospitalized": "hospitalized_new",
    "census_hospitalized": "hospitalized",
}

## Set up Penn CHIME model

In [ ]:
p = Parameters(
    current_hospitalized=69,
    date_first_hospitalized=date(2020, 3, 7),
    doubling_time=4.0,
    hospitalized=Disposition.create(days=7, rate=0.025),
    icu=Disposition.create(days=9, rate=0.0075),
    infectious_days=14,
    market_share=0.15,
    n_days=100,
    population=3600000,
    recovered=0,
    relative_contact_rate=0.3,
    ventilated=Disposition.create(days=10, rate=0.005),
)

p.doubling_time = None
simsir = Sir(p)

## Tests

### Check that model agrees with Penn CHIME if no policies are in place

Calculate S, I, H, R for no policies

In [ ]:
n_days = simsir.raw_df.day.max() - simsir.raw_df.day.min() + 1

policies = [(simsir.beta, n_days)]
raw = sim_sir(
    simsir.susceptible,
    simsir.infected,
    p.recovered,
    simsir.gamma,
    -simsir.i_day,
    policies,
)


calculate_dispositions(raw, simsir.rates, market_share=p.market_share)
calculate_admits(raw, simsir.rates)
calculate_census(raw, simsir.days)

raw_df = DataFrame(raw)

day0 = raw_df.iloc[0].fillna(0)

raw_df.head()

Compute values using new fit function

In [ ]:
pars = {
    "beta_i": simsir.beta * p.population,
    "gamma_i": simsir.gamma,
    "initial_susceptible": day0.susceptible,
    "initial_infected": day0.infected,
    "initial_hospitalized": day0.hospitalized,
    "initial_recovered": day0.recovered,
    "hospitalization_rate": simsir.rates["hospitalized"] * p.market_share,
}
x = {
    "n_iter": raw_df.shape[0],
    "length_of_stay": p.dispositions["hospitalized"].days,
}


f = FitFcn(sir_step)
y = f(x, pars)
y.head()

Check that difference is consistent with zero

In [ ]:
diff = (raw_df.rename(columns=COLUMN_MAP) - y)[COLS_TO_COMPARE]
mean = diff.mean()
sdev = diff.std()
assert (mean.abs() < 2 * sdev + 1.0e-7).all()
DataFrame([mean, sdev], index=["mean", "sdev"]).T

### Check that model agrees with Penn CHIME if no policies are in place

Now compare against Penn CHIME with active social distancing policies.

This repo's SIR function takes a different input for social distancing policies to allow fitting later on.
The policies are implemented as a function which returns an array of betas

In [ ]:
POLICIES = simsir.gen_policy(p)

day0 = simsir.raw_df.iloc[0]
total = day0.susceptible + day0.infected + day0.recovered


def beta_i_fcn(x_iter, **kwargs):
    out = zeros(len(x_iter))
    ii = 0
    for beta, n_days in POLICIES:
        for _ in range(n_days):
            out[ii] = beta * p.population
            ii += 1

    return out

In [ ]:
pars = {
    "beta_i": None,
    "gamma_i": simsir.gamma,
    "initial_susceptible": day0.susceptible,
    "initial_infected": day0.infected,
    "initial_hospitalized": day0.hospitalized,
    "initial_recovered": day0.recovered,
    "hospitalization_rate": simsir.rates["hospitalized"] * p.market_share,
}
x = {
    "n_iter": simsir.raw_df.shape[0],
    "length_of_stay": p.dispositions["hospitalized"].days,
}


f = FitFcn(sir_step, beta_i_fcn=beta_i_fcn,)
y = f(x, pars)
y.head()

In [ ]:
diff = (simsir.raw_df.rename(columns=COLUMN_MAP) - y)[COLS_TO_COMPARE].fillna(0)
mean = diff.mean()
sdev = diff.std()
assert (mean.abs() < 2 * sdev + 1.0e-7).all()
DataFrame([mean, sdev], index=["mean", "sdev"]).T